In [ ]:
!pip install datasets
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.5 MB/s eta 0:00:00


Посмотрим на то, как модель анализирует тональность.

In [ ]:
from datasets import load_dataset

dataset = load_dataset('cardiffnlp/tweet_sentiment_multilingual', 'spanish')
dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.28k [00:00<?, ?B/s]

tweet_sentiment_multilingual.py:   0%|          | 0.00/4.14k [00:00<?, ?B/s]

spanish%2Ftrain%2F0000.parquet:   0%|          | 0.00/120k [00:00<?, ?B/s]

spanish%2Fvalidation%2F0000.parquet:   0%|          | 0.00/23.6k [00:00<?, ?B/s]

spanish%2Ftest%2F0000.parquet:   0%|          | 0.00/58.0k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1839 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/324 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/870 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1839
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 324
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 870
    })
})

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/310 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/486k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

In [ ]:
def make_dataset(data, tokenizer):
    answer = []
    for i, item in enumerate(data):
        if item["label"] == 1:
          continue
        result = tokenizer(item["text"])
        result["labels"] = 1 if item["label"] == 2 else item["label"]
        # result["index"] = i
        answer.append(result)
    return answer

In [ ]:
train_dataset = make_dataset(dataset['train'], tokenizer)
dev_dataset = make_dataset(dataset['validation'], tokenizer)
test_dataset = make_dataset(dataset['test'], tokenizer)

In [ ]:
for k, v in train_dataset[1].items():
    print(k, v)

input_ids [4, 1202, 1009, 1553, 3416, 28927, 5]
token_type_ids [0, 0, 0, 0, 0, 0, 0]
attention_mask [1, 1, 1, 1, 1, 1, 1]
labels 1


In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer
from transformers.optimization import AdamW

model = AutoModelForSequenceClassification.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased', force_download=True, num_labels=2)
optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)
training_args = TrainingArguments(output_dir='/content/checkpoints',
                                  evaluation_strategy="epoch", save_strategy='epoch', num_train_epochs=10,
                                  load_best_model_at_end=True, disable_tqdm=False,
                                  per_device_train_batch_size=4, warmup_ratio=0.1,
                                  gradient_accumulation_steps=4, metric_for_best_model="Accuracy")

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    pred_labels = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, pred_labels)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, pred_labels, average='macro')
    return {'Accuracy': accuracy * 100, 'Precision': precision * 100, 'Recall': recall * 100, 'F1': f1 * 100}

In [ ]:
trainer = Trainer(
    model=model,
    optimizers=(optimizer, None), # optimizers=(optimizer, scheduler)
    args=training_args,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer), # добавляет паддинг в более короткие элементы батча
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sofiesolve (sofiesolve-msu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,No log,0.472816,84.259259,84.271012,84.259259,84.257910
1,No log,0.316881,88.425926,88.828727,88.425926,88.395831
2,No log,0.308121,86.574074,86.829837,86.574074,86.550725
3,No log,0.477819,86.111111,86.562500,86.111111,86.068111
4,No log,0.494362,88.888889,88.942308,88.888889,88.885077
5,No log,0.518558,89.814815,90.159114,89.814815,89.792938
6,0.235500,0.620149,87.500000,87.503215,87.500000,87.499732
7,0.235500,0.648818,87.962963,87.962963,87.962963,87.962963
8,0.235500,0.628481,87.962963,87.975986,87.962963,87.961931
9,0.235500,0.618524,89.351852,89.436378,89.351852,89.346143


TrainOutput(global_step=760, training_loss=0.15850241372459814, metrics={'train_runtime': 310.7277, 'train_samples_per_second': 39.456, 'train_steps_per_second': 2.446, 'total_flos': 197690958110880.0, 'train_loss': 0.15850241372459814, 'epoch': 9.990228013029316})

In [ ]:
trainer = Trainer(
    model=model,
    optimizers=(optimizer, None), # optimizers=(optimizer, scheduler)
    args=training_args,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer), # добавляет паддинг в более короткие элементы батча
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics)

trainer.evaluate()

{'eval_loss': 0.7353566288948059,
 'eval_model_preparation_time': 0.0044,
 'eval_Accuracy': 84.82758620689656,
 'eval_Precision': 85.25687322451732,
 'eval_Recall': 84.82758620689656,
 'eval_F1': 84.7812604360479,
 'eval_runtime': 1.3898,
 'eval_samples_per_second': 417.329,
 'eval_steps_per_second': 52.526}

In [ ]:
import numpy as np
import scipy
import warnings
warnings.filterwarnings('ignore')

def eval(s):
    data_samples = [{"text": s.lower(), "label": 0}]
    dataset = make_dataset(data_samples, tokenizer)
    classes = [0, 1]
    predictions = trainer.predict(dataset)
    probs = scipy.special.softmax(predictions.predictions, axis=-1)
    answer = [{"positive": bool(classes[np.argmax(elem)]), "probs": elem} for elem in probs]
    return answer

In [ ]:
eval('Que bueno!')

[{'positive': True, 'probs': array([0.00101912, 0.9989808 ], dtype=float32)}]

In [ ]:
eval('Que dolor!')

[{'positive': False, 'probs': array([0.99171406, 0.00828595], dtype=float32)}]

In [ ]:
eval('¡Me encanta el sol de la mañana!')

[{'positive': True, 'probs': array([0.0010724 , 0.99892753], dtype=float32)}]

In [ ]:
eval('Siempre disfruto de un buen libro con una taza de té.')

[{'positive': True, 'probs': array([0.00115178, 0.9988482 ], dtype=float32)}]

In [ ]:
eval('Este café está delicioso, ¡gracias por prepararlo!')

[{'positive': True,
  'probs': array([9.079143e-04, 9.990921e-01], dtype=float32)}]

In [ ]:
eval('No soporto el ruido constante de la construcción.')

[{'positive': False, 'probs': array([0.99823254, 0.00176751], dtype=float32)}]

In [ ]:
eval('Este lugar me parece frío y desagradable.')

[{'positive': False, 'probs': array([0.9982181 , 0.00178184], dtype=float32)}]

In [ ]:
eval('Hemos perdido el tren por llegar tarde.')

[{'positive': False, 'probs': array([0.9974733 , 0.00252665], dtype=float32)}]

Предложения с очевидным смыслом модель оценивает с легкостью.

In [ ]:
eval('Terminé de leer el libro que me recomendaste.')

[{'positive': True, 'probs': array([0.0021363, 0.9978637], dtype=float32)}]

У этой фразы в определенном контексте мог бы быть и негативный смысл. Но в большинстве контекстов она скорее нейтральная. Но так как у нас нет такой категории, модели приходится выбирать из существующих двух.

In [ ]:
eval('El clima hoy es… interesante.')

[{'positive': True, 'probs': array([0.00137173, 0.99862826], dtype=float32)}]

Сарказм модель не понимает.

In [ ]:
eval('Finalmente tengo tiempo libre.')

[{'positive': True, 'probs': array([0.00175437, 0.9982456 ], dtype=float32)}]

В некоторых контекстах (увольнение) может иметь негативный окрас, но чаще, как и оценено моделью, положительное.

In [ ]:
eval('Tu forma de resolver problemas es única.')

[{'positive': True, 'probs': array([0.00220468, 0.99779534], dtype=float32)}]

И опять же можно сказать, что сарказм модель не понимает.

Итак, модель справляется с более очевидными предложениями, однако, она совсем, в упор, не понимает сарказма и, вероятно (но не точно), смотрит на ключевые слова.